# Part I: Model Building & Training

In [1]:
import pandas as pd
import numpy as np
import os
import pickle
import json
import gradio as gr

from sklearn.metrics import accuracy_score, confusion_matrix

from numpy import mean, std

from matplotlib import pyplot
import tensorflow
from tensorflow import keras
from keras.datasets import mnist
from keras.models import load_model
from kerastuner.tuners import RandomSearch
from kerastuner.engine.hyperparameters import HyperParameters

from tensorflow.keras.models import Sequential
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.layers import Conv2D, Dense, MaxPooling2D, Flatten, Dropout, Reshape
from tensorflow.keras.optimizers import SGD
from tensorflow.keras.wrappers.scikit_learn import KerasClassifier

Using TensorFlow backend.


### Loading Data

In [16]:
def data_loading():
    # Loading dataset from MNIST package & assign to train test variables
    (trainx, trainy), (testx, testy) = mnist.load_data()
    # Reshaping pixels into single array for DL model input layer
    trainx = trainx.reshape((-1, 28, 28, 1))
    testx = testx.reshape((-1, 28, 28, 1))
    # One Hot Encoding (OHE) for labels
    trainy = to_categorical(trainy)
    
    return trainx, trainy, testx, testy

In [17]:
trainx, trainy, testx, testy = data_loading()

### Preparation & Normalize Pixels (only for train & test data)

In [4]:
def pixels_transforming():
    # convert from integers to floats
    train_norm = trainx.astype('float32')
    test_norm = testx.astype('float32')
    # normalize to range 0-1
    train_norm = train_norm / 255.0
    test_norm = test_norm / 255.0
    # return normalized images
    
    return train_norm, test_norm

In [5]:
trainx, testx = pixels_transforming()

### 1. Deep Neural Network (DNN)

In [15]:
# Dropout reqularization is a computationally cheap way to regularize deep neural network. Also reduces overfitting.

def model_building(hp):
    model = Sequential()
    
    # input layer
    # VGG Block 1 (input format 28x28 pixels) + padding
    model.add(Conv2D(hp.Int('inputs',
                            min_value=32,
                            max_value=256,
                            step=32),
                     (3, 3), padding='same',
                     activation='relu',
                     kernel_initializer='he_uniform', 
                     input_shape=(28, 28, 1)))
    model.add(MaxPooling2D((2, 2)))
    model.add(Dropout(hp.Choice('drops', [0.1,0.2,0.3])))
#     # VGG Block 2 + padding (increased filters)
#     model.add(Conv2D(64, (3, 3), padding='same', activation='relu', kernel_initializer='he_uniform'))
#     model.add(MaxPooling2D((2, 2)))
#     model.add(Dropout(0.2))
#     # VGG Block 3 + padding (increased filters)
#     model.add(Conv2D(128, (3, 3), activation='relu', kernel_initializer='he_uniform', padding='same'))
#     model.add(MaxPooling2D((2, 2)))
#     model.add(Dropout(0.2))
    model.add(Flatten())
    
    # hidden layers
    for i in range(hp.Int('num_layers', min_value=1, max_value=10, step=1)):
        model.add(Dense(hp.Int('num_nodes',
                               min_value=32,
                               max_value=256,
                               step=32),
                        activation='relu',
                        kernel_initializer='he_uniform'))
        model.add(Dropout(hp.Choice('drops', [0.1,0.2,0.3])))
    
    # output layer: if binary classification, use 1 layer and sigmoid activation
    model.add(Dense(10, activation='softmax'))
    
    # compile model
    # if binary classification, loss is binary_crossentropy
    model.compile(optimizer='adam',
                  loss='categorical_crossentropy',
                  metrics=['accuracy'])
    
    return model

### 2. Hyperparameter Tuning (using GridSearchCV)

In [6]:
def model_tuning():
    tuner = RandomSearch(model_building,
                     objective='val_accuracy',
                     max_trials=5,
                     directory=os.path.normpath('C:/MNIST Digits Classification'),
                     executions_per_trial=3,
                     project_name='MNIST Digit Keras Tuner')
    tuner.search_space_summary()
    # if doesn't run, please delete pre-load runs from above directory
    tuner.search(trainx, trainy, epochs=5, batch_size=64, validation_split=0.2, shuffle=True)
    return tuner

In [ ]:
tuner = model_tuning()

### 3. Best Model Building

##### Define DNN model network using best parameters from Keras Tuner & continue with model prediction.

In [58]:
def best_model():
    model = Sequential()
    
    # input layer
    # VGG Block 1 (input format 28x28 pixels) + padding
    model.add(Conv2D(192, (3, 3), padding='same', activation='relu', kernel_initializer='he_uniform', input_shape=(28, 28, 1)))
    model.add(MaxPooling2D((2, 2)))
    model.add(Dropout(0.2))
    model.add(Flatten())
    
    # hidden layers
    model.add(Dense(128, activation='relu', kernel_initializer='he_uniform'))
    model.add(Dense(128, activation='relu', kernel_initializer='he_uniform'))
    model.add(Dense(128, activation='relu', kernel_initializer='he_uniform'))
    model.add(Dense(128, activation='relu', kernel_initializer='he_uniform'))
    model.add(Dense(128, activation='relu', kernel_initializer='he_uniform'))
    model.add(Dense(128, activation='relu', kernel_initializer='he_uniform'))
    model.add(Dropout(0.2))
    
    # output layer: if binary classification, use 1 layer and sigmoid activation
    model.add(Dense(10, activation='softmax'))
    
    # compile model
    # if binary classification, loss is binary_crossentropy
    model.compile(optimizer='adam',
                  loss='categorical_crossentropy',
                  metrics=['accuracy'])
    
    model.fit(trainx, trainy, validation_split=0.2, verbose=0)
    
    #save best model
    model.save('final_model_deploy.h5')
    
    return model

In [7]:
final_model = best_model()

# Part II: Model Testing (on test data)

In [62]:
def model_testing():
    model_pred = final_model.predict_classes(testx)
    print('Accuracy :', accuracy_score(testy, model_pred), '\n')
    print('Confusion Matrix :\n', confusion_matrix(testy, model_pred))

In [9]:
model_testing()

C:\ProgramData\Anaconda3\lib\site-packages\tensorflow\python\keras\engine\sequential.py:450: UserWarning: `model.predict_classes()` is deprecated and will be removed after 2021-01-01. Please use instead:* `np.argmax(model.predict(x), axis=-1)`,   if your model does multi-class classification   (e.g. if it uses a `softmax` last-layer activation).* `(model.predict(x) > 0.5).astype("int32")`,   if your model does binary classification   (e.g. if it uses a `sigmoid` last-layer activation).
  warnings.warn('`model.predict_classes()` is deprecated and '


Accuracy : 0.9703 

Confusion Matrix :
 [[ 966    0    0    0    0    1    3    1    9    0]
 [   0 1120    2    5    0    0    0    1    7    0]
 [   2    0  992   11    4    0    2    9   11    1]
 [   0    0    3  979    0   12    0    3    2   11]
 [   0    0    1    0  936    0    6   13   12   14]
 [   5    0    0    4    0  852    9    1    3   18]
 [  11    3    0    0    0    1  934    1    8    0]
 [   0    0    8    8    0    0    0 1004    5    3]
 [   3    0    3    2    1    2    1    6  954    2]
 [   4    3    0    1    3    4    1   17   10  966]]


# Part III: Model Live Prediction

In [2]:
model = tensorflow.keras.models.load_model("final_model_deploy.h5")

def recognize_digit(image):
    image = image.reshape(1, -1)  # add a batch dimension
    prediction = model.predict(image).tolist()[0]
    return {str(i): prediction[i] for i in range(10)}

output_component = gr.outputs.Label(num_top_classes=3)

gr.Interface(fn=recognize_digit, 
             inputs="sketchpad", 
             outputs=output_component,
             live=True,
             title="MNIST Sketchpad",
             description="Draw a number 0 through 9 on the sketchpad, and click submit to see the model's predictions. Model trained on the MNIST dataset.",
             thumbnail="https://raw.githubusercontent.com/gradio-app/real-time-mnist/master/thumbnail2.png").launch();

Running locally at: http://127.0.0.1:7860/
To create a public link, set `share=True` in `launch()`.
Interface loading below...
